In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark import functions as SF

from dotenv import load_dotenv
import os



load_dotenv()
SF_USER = os.getenv("SF_USER")
SF_PASSWORD = os.getenv("SF_PASSWORD")
SF_ACCOUNT = os.getenv("SF_ACCOUNT")
OPENROUTESERVICE_API_KEY = os.getenv("OPENROUTESERVICE_API_KEY")


connection_parameters = {
  "account": SF_ACCOUNT,
  "user": SF_USER,
  "password": SF_PASSWORD,
}
new_session = Session.builder.configs(connection_parameters).create()

In [5]:
db = "PANUCCIS_PIZZA"
schema = "POS_MAIN"
WH = "PANUCCIS_PIZZA_WH"
table_stores = "STORES"
table_customers = "CUSTOMERS"
table_orders = "ORDERS_HEADER"


new_session.use_database(db)
new_session.use_schema(schema)
new_session.use_warehouse(WH)

In [ ]:
# Push secret to snowflake
secret_name = 'OPENROUTESERVICE_API_KEY'
secret_value = OPENROUTESERVICE_API_KEY

sql_query = f"""
CREATE OR REPLACE SECRET {secret_name}
  TYPE = GENERIC_STRING
  SECRET_STRING = '{secret_value}'
  COMMENT = 'API key for OpenRouteService'
"""

# Execute the query
new_session.sql(sql_query).collect()

[Row(status='Secret OPENROUTESERVICE_API_KEY successfully created.')]

In [21]:
new_session.sql("show secrets").show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"created_on"                      |"name"                    |"schema_name"  |"database_name"  |"owner"       |"comment"                     |"secret_type"   |"oauth_scopes"  |"owner_role_type"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2025-04-07 18:17:30.087000-07:00  |OPENROUTESERVICE_API_KEY  |POS_MAIN       |PANUCCIS_PIZZA   |ACCOUNTADMIN  |API key for OpenRouteService  |GENERIC_STRING  |NULL            |ROLE               |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [24]:
def get_isochrones_for_time(lat, lon, time_range_in_minutes):
    """
    Get isochrones for a given latitude and longitude with a specified time range.
    """
    import requests
    body = {
        "locations": [[lon, lat]],
        "range": [t*60 for t in time_range_in_minutes], # Convert minutes to seconds
        "range_type": "time"
    }

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': OPENROUTESERVICE_API_KEY,
        'Content-Type': 'application/json; charset=utf-8'
    }

    response = requests.post('https://api.openrouteservice.org/v2/isochrones/driving-car', json=body, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")
    

# Example usage
lat = 33.18667
lon = -96.8761
time_range_in_minutes = [ 5,10]  # in minutes



result = get_isochrones_for_time(lat, lon, time_range_in_minutes)


In [25]:
result

{'type': 'FeatureCollection',
 'bbox': [-96.929677, 33.138383, -96.807631, 33.235216],
 'features': [{'type': 'Feature',
   'properties': {'group_index': 0,
    'value': 300.0,
    'center': [-96.87628024381381, 33.186757316172816]},
   'geometry': {'coordinates': [[[-96.901391, 33.186488],
      [-96.900955, 33.186278],
      [-96.899327, 33.185748],
      [-96.895104, 33.184004],
      [-96.893961, 33.18346],
      [-96.887891, 33.178436],
      [-96.886904, 33.176637],
      [-96.886038, 33.175892],
      [-96.884348, 33.175625],
      [-96.878966, 33.174906],
      [-96.878718, 33.174908],
      [-96.873379, 33.175776],
      [-96.867326, 33.174858],
      [-96.86522, 33.174065],
      [-96.865184, 33.174076],
      [-96.864026, 33.176295],
      [-96.863991, 33.176848],
      [-96.86332, 33.18031],
      [-96.859438, 33.185654],
      [-96.852284, 33.187945],
      [-96.85044, 33.187925],
      [-96.846497, 33.187877],
      [-96.846394, 33.187875],
      [-96.844676, 33.189345],
